In [1]:
import psycopg2 as ps
import pandas as pd
import numpy as np

import psycopg2 as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats

%matplotlib inline

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 8, 8
plt.rcParams['font.size'] = 12
sns.set_style('darkgrid')

sns.set(style="darkgrid")

In [2]:
import scipy.stats.distributions as dist

def z_test_2_sample_proportions(x_control, x_test, n_control, n_test, alpha):
    '''
    Calculate the test statistic for a z-test on 2 proportions from independent samples
    x1, x2: number of successes in group 1 and 2
    n1, n2: total number of observations in group 1 and 2
    Returns: test statistic (z), and p-value 
    '''
    avg_p = (x_control + x_test) / (n_control + n_test)
    z_val = (x_control/n_control - x_test/n_test) / np.sqrt(avg_p * (1-avg_p) * (1/n_control + 1/n_test))
    z_prob = dist.norm.cdf(-np.abs(z_val))
    
    if 2*z_prob < alpha:
        print(f'p-value = {2*z_prob}, нулевая гипотеза отвергается - изменение статистически значимо')
        print(f'Изменение теста по сравнению с контролем составило {(x_test/n_test - x_control/n_control) * 100/(x_control/n_control)}%')
        print(f'Среднее в контроле - {round(x_control/n_control,3)}, среднее в тесте - {round(x_test/n_test,3)}')
    else:
        print(f'p-value = {2*z_prob}, нулевая гипотеза не отвергается - изменение статистически не значимо')
        print(f'Изменение теста по сравнению с контролем составило {(x_test/n_test - x_control/n_control) * 100/(x_control/n_control)}%')
        print(f'Среднее в контроле - {round(x_control/n_control,3)}, среднее в тесте - {round(x_test/n_test,3)}')

In [3]:
def ttest(control, test, alpha):
    res = stats.ttest_ind(a=control, b=test, equal_var=False)
    
    if res.pvalue < alpha:
        print(f'p-value = {round(res.pvalue, 4)}, нулевая гипотеза отвергается - изменение статистически значимо')
        print(f'Изменение теста по сравнению с контролем составило {round((test.mean() - control.mean()) * 100/(control.mean()),3)}%')
        print(f'Среднее в контроле - {round(control.mean(),3)}, среднее в тесте - {round(test.mean(),3)}')
    else:
        print(f'p-value = {round(res.pvalue, 4)}, нулевая гипотеза не отвергается - изменение статистически не значимо')
        print(f'Изменение теста по сравнению с контролем составило {round((test.mean() - control.mean()) * 100/(control.mean()),3)}%')
        print(f'Среднее в контроле - {round(control.mean(),3)}, среднее в тесте - {round(test.mean(),3)}')

In [4]:
conn = ps.connect(host="46.34.136.91", port = 25432, database="postgres", 
                  user="aaa_student", password="student_@@@_2022")

<center><h2> Анализ результатов </center></h2>

Все метрики посчитаны как поюзерные а не ratio, чтобы обеспечить независимость наблюдений. Во всех проверках гипотез используется поправка Бонферони, где уровень значимости каждой гипотезы определяется как $\frac{\alpha}{m}$, где $m$ - количество проверяемых гипотез. В данном случае $m = 2,\alpha = 5%$, то есть скорректированное значение уровня значимости будет равняться 2.5%

<center><h3> Верифицировавшихся за период проведения эксперимента пользователи - целевая </center></h3>

In [5]:
df_ver_number = pd.read_sql("""with verificated as (select distinct user_id,
                                                           1 as ver_done
                                                    from PUBLIC.services_verification_discounts
                                                    where discount_day  between '2021-09-15' and '2023-10-05')

                              select v.user_id,
                                     v.ver_done,
                                     sves.experiment_group 
                              from PUBLIC.services_verification_experiment_segment sves 
                              left join verificated v
                                  on v.user_id = sves.user_id """, con=conn)

In [6]:
df_ver_number = df_ver_number.fillna(0)

In [7]:
df_30_discount = df_ver_number[df_ver_number['experiment_group'] == '30_discount']['ver_done']
df_60_discount = df_ver_number[df_ver_number['experiment_group'] == '60_discount']['ver_done']
df_no_discount = df_ver_number[df_ver_number['experiment_group'] == 'no_discount']['ver_done']

In [8]:
x_control = df_no_discount.sum()
x_test = df_30_discount.sum()

n_control = len(df_no_discount)
n_test = len(df_30_discount)

alpha = 0.025

z_test_2_sample_proportions(x_control, x_test, n_control, n_test, alpha)

p-value = 9.703902026399639e-09, нулевая гипотеза отвергается - изменение статистически значимо
Изменение теста по сравнению с контролем составило 9.907089093964023%
Среднее в контроле - 0.027, среднее в тесте - 0.03


In [9]:
x_control = df_no_discount.sum()
x_test = df_60_discount.sum()

n_control = len(df_no_discount)
n_test = len(df_60_discount)

alpha = 0.025

z_test_2_sample_proportions(x_control, x_test, n_control, n_test, alpha)

p-value = 1.1864541252784637e-35, нулевая гипотеза отвергается - изменение статистически значимо
Изменение теста по сравнению с контролем составило 22.127980498278035%
Среднее в контроле - 0.027, среднее в тесте - 0.033


Главная гипотеза подтвердилась, покупатели действительно стали верифицироваться чаще при предоставлении скидки. Причем в группе со скидкой 60% эта разница намного выше.

<center><h3> Количество купленных услуг по продвижению типа 3 - целевая </center></h3>

In [10]:
df_cnt_buy = pd.read_sql("""with buy as (select upi.user_id,
                                            count(upi.item_id) as cnt
                                     from PUBLIC.user_payments_info upi
                                     left join PUBLIC.user_item_info uii 
                                         on uii.item_id  = upi.item_id 
                                     where upi.event_time between '2021-09-15' and '2023-10-05'
                                         and upi.transaction_type  = '3'
                                         and uii.item_vertical  = '1'
                                     group by upi.user_id)

select b.user_id,
       b.cnt,
       sves.experiment_group 
from PUBLIC.services_verification_experiment_segment sves 
left join buy b
    on b.user_id = sves.user_id 
where b.user_id in (select user_id 
                    from PUBLIC.services_verification_discounts
                    where  discount_day  between '2021-09-15' and '2023-10-05')""", con=conn)

In [11]:
df_cnt_buy = df_cnt_buy.fillna(0)

In [12]:
df_30_discount = df_cnt_buy[df_cnt_buy['experiment_group'] == '30_discount']['cnt']
df_60_discount = df_cnt_buy[df_cnt_buy['experiment_group'] == '60_discount']['cnt']
df_no_discount = df_cnt_buy[df_cnt_buy['experiment_group'] == 'no_discount']['cnt']

In [13]:
 ttest(df_no_discount, df_30_discount, 0.025)

p-value = 0.5889, нулевая гипотеза не отвергается - изменение статистически не значимо
Изменение теста по сравнению с контролем составило -4.795%
Среднее в контроле - 5.977, среднее в тесте - 5.691


In [14]:
 ttest(df_no_discount, df_60_discount, 0.025)

p-value = 0.4605, нулевая гипотеза не отвергается - изменение статистически не значимо
Изменение теста по сравнению с контролем составило -5.572%
Среднее в контроле - 5.977, среднее в тесте - 5.644



Изменение в обоих случаях незначимо, услуги 3-го типа покупают одинаково.

<center><h3> Количество созданных за период эксперимента объявлений - дополнительная </center></h3>

In [15]:
df_adv_number = pd.read_sql("""with adverts as (select user_id,
                                                       count(item_id) as adv_count
                                                from PUBLIC.user_item_info
                                                where item_creation_time  between '2021-09-15' and '2023-10-05'
                                                      and item_vertical = '1'
                                                group by user_id)

                               select a.user_id,
                                      a.adv_count,
                                      sves.experiment_group 
                               from PUBLIC.services_verification_experiment_segment sves 
                               left join adverts a
                                   on a.user_id = sves.user_id 
                               where a.user_id in (select user_id 
                                                   from PUBLIC.services_verification_discounts
                                                    where  discount_day  between '2021-09-15' and '2023-10-05')""", con=conn)

In [16]:
df_adv_number = df_adv_number.fillna(0)

In [17]:
df_30_discount = df_adv_number[df_adv_number['experiment_group'] == '30_discount']['adv_count']
df_60_discount = df_adv_number[df_adv_number['experiment_group'] == '60_discount']['adv_count']
df_no_discount = df_adv_number[df_adv_number['experiment_group'] == 'no_discount']['adv_count']

In [18]:
 ttest(df_no_discount, df_30_discount, 0.025)

p-value = 0.4084, нулевая гипотеза не отвергается - изменение статистически не значимо
Изменение теста по сравнению с контролем составило 13.308%
Среднее в контроле - 3.444, среднее в тесте - 3.902


In [19]:
 ttest(df_no_discount, df_60_discount, 0.025)

p-value = 0.258, нулевая гипотеза не отвергается - изменение статистически не значимо
Изменение теста по сравнению с контролем составило -11.25%
Среднее в контроле - 3.444, среднее в тесте - 3.056



Гипотеза не подствердилась, во всех группах количество созданных объявлений находится на одном и том же уровне.

<center><h3>Количество купленных услуг по продвижению любого типа </center></h3>

In [20]:
df_cnt_buy_all_types = pd.read_sql("""with buy as (select upi.user_id,
                                            count(upi.item_id) as cnt
                                     from PUBLIC.user_payments_info upi
                                     left join PUBLIC.user_item_info uii 
                                         on uii.item_id  = upi.item_id 
                                     where upi.event_time between '2021-09-15' and '2023-10-05'
                                         and uii.item_vertical  = '1'
                                     group by upi.user_id)

select b.user_id,
       b.cnt,
       sves.experiment_group 
from PUBLIC.services_verification_experiment_segment sves 
left join buy b
    on b.user_id = sves.user_id 
where b.user_id in (select user_id 
                    from PUBLIC.services_verification_discounts
                    where  discount_day  between '2021-09-15' and '2023-10-05')""", con=conn)

In [21]:
df_cnt_buy_all_types = df_cnt_buy_all_types.fillna(0)

In [22]:
df_30_discount = df_cnt_buy_all_types[df_cnt_buy_all_types['experiment_group'] == '30_discount']['cnt']
df_60_discount = df_cnt_buy_all_types[df_cnt_buy_all_types['experiment_group'] == '60_discount']['cnt']
df_no_discount = df_cnt_buy_all_types[df_cnt_buy_all_types['experiment_group'] == 'no_discount']['cnt']

In [23]:
 ttest(df_no_discount, df_30_discount, 0.025)

p-value = 0.8736, нулевая гипотеза не отвергается - изменение статистически не значимо
Изменение теста по сравнению с контролем составило -1.185%
Среднее в контроле - 9.031, среднее в тесте - 8.924


In [24]:
 ttest(df_no_discount, df_60_discount, 0.025)

p-value = 0.274, нулевая гипотеза не отвергается - изменение статистически не значимо
Изменение теста по сравнению с контролем составило -6.658%
Среднее в контроле - 9.031, среднее в тесте - 8.43


Что уже и ожидалось, количество покупок услуг любого типа тоже не отличается в зависимости от группы

<center><h3> Доходы от комиссии за рекламные объявления </center></h3>

In [25]:
df_buy = pd.read_sql("""with buy as (select upi.user_id,
                                            sum(upi.amount_net) as amount
                                     from PUBLIC.user_payments_info upi
                                     left join PUBLIC.user_item_info uii 
                                         on uii.item_id  = upi.item_id 
                                     where upi.event_time between '2021-09-15' and '2023-10-05'
                                         and upi.transaction_type  = '3'
                                         and uii.item_vertical  = '1'
                                     group by upi.user_id)

                         select b.user_id,
                                case
                                   when experiment_group = '30_discount' then  b.amount*0.7
                                   when experiment_group = '60_discount' then  b.amount*0.4
                                   else b.amount
                                end as amount_final,
                                sves.experiment_group 
                         from PUBLIC.services_verification_experiment_segment sves 
                         left join buy b
                             on b.user_id = sves.user_id 
                         where b.user_id in (select user_id 
                                             from PUBLIC.services_verification_discounts
                                             where  discount_day  between '2021-09-15' and '2023-10-05')""", con=conn)

In [26]:
df_buy = df_buy.fillna(0)

In [27]:
df_30_discount = df_buy[df_buy['experiment_group'] == '30_discount']['amount_final']
df_60_discount = df_buy[df_buy['experiment_group'] == '60_discount']['amount_final']
df_no_discount = df_buy[df_buy['experiment_group'] == 'no_discount']['amount_final']

In [28]:
 ttest(df_no_discount, df_30_discount, 0.025)

p-value = 0.0, нулевая гипотеза отвергается - изменение статистически значимо
Изменение теста по сравнению с контролем составило -30.465%
Среднее в контроле - 328.592, среднее в тесте - 228.485


In [29]:
 ttest(df_no_discount, df_60_discount, 0.025)

p-value = 0.0, нулевая гипотеза отвергается - изменение статистически значимо
Изменение теста по сравнению с контролем составило -63.168%
Среднее в контроле - 328.592, среднее в тесте - 121.026


Полученные доходы снизились в обоих тестовых группах. При этом они снизились примерно на такое же количество процентов, на сколько была сделана скидка. Это логично, так как как количество продаж не изменилось.

<center><h2> Выводы и рекомендации </center></h2>

Данный тест нельзя считать успешным, так как просела одна из главных метрик - доходы за продажи от услуг 3-го типа.

Однако следует сделать несколько оговорок:

1. Скидка выдвется лишь на 2 дня, а покупка услуг продвижения - не каждодневная. Вполне возможно, что продавцы купили эту услугу по одному разу, и эта гипотеза может полностью объяснить такое падение доходов и отсутствие изменений спроса на услуги 3-го типа.

2. Возможно, доходы от услуг 3-го типа не имеют такой сильный вклад в общие доходы, и метрика общего количества доходов изменилась не стат. значимо

В дальнейшем предлагается использовать другие методы мотивации для продавцов. Например, использовать меньший размер скидки. Также предлагается использовать больший срок для эксперимента.

Нельзя сказать что эксперимент был плохим. Верификация - одноразовое событие, котрое остается с продавцом. В ходе этого эксперимента удалось увеличить % верифицировавшихся, чего бы не произошло без теста